In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from glob import glob
import numpy as np

Using TensorFlow backend.


In [2]:
batch_size = 16

# this is the configuration for training set 
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# For testing only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator, will read pictures found in subfolers of 'data/train'
data_generator = train_datagen.flow_from_directory(
        './train/',  # this is the target directory
        target_size=(64, 64),  # all images will be resized to 150x150
        color_mode = 'rgb', # rgb, grayscale
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels


Found 24000 images belonging to 2 classes.


In [3]:
X = list()
y = list()
batch_index = 0

while batch_index <= data_generator.batch_index:
    data = data_generator.next()
    for batch_data_X,batch_data_y in zip(data[0],data[1]):
        X.append(batch_data_X)
        y.append(batch_data_y)
    batch_index = batch_index + 1
    #if batch_index>1000:
    #    break
del data_generator
X = np.asarray(X)
y = to_categorical(y, num_classes=2)

In [4]:
X.shape

(24000, 64, 64, 3)

In [5]:
n_imgs = X.shape[0]
n_cols = X.shape[1]*X.shape[2]

In [6]:
model = Sequential()
model.add(Conv2D(32,kernel_size=(3, 3), activation= 'relu',  input_shape=X.shape[1:]))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Flatten())

model.add(Dense(150,activation='relu'))

model.add(Dropout(0.2))
model.add(Dense(2,activation='softmax',input_shape= (n_cols,) ))

ValueError: Negative dimension size caused by subtracting 3 from 1 for 'max_pooling2d_1/MaxPool' (op: 'MaxPool') with input shapes: [?,1,62,32].

In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [9]:
my_callback = EarlyStopping(patience=5)

In [ ]:
model.fit(X, y,
        batch_size=150, 
        epochs=50,
        validation_split=0.2,
        callbacks =  [my_callback])
model.save_weights('first_try.h5')  # always save your weights after training or during training

Train on 20000 samples, validate on 5000 samples
Epoch 1/50
20000/20000 [==============================] - 65s 3ms/step - loss: 0.6763 - acc: 0.6097 - val_loss: 0.6135 - val_acc: 0.6634
Epoch 2/50
20000/20000 [==============================] - 62s 3ms/step - loss: 0.5838 - acc: 0.6949 - val_loss: 0.5695 - val_acc: 0.7036
Epoch 3/50
20000/20000 [==============================] - 63s 3ms/step - loss: 0.5422 - acc: 0.7262 - val_loss: 0.5461 - val_acc: 0.7276
Epoch 4/50
20000/20000 [==============================] - 62s 3ms/step - loss: 0.5204 - acc: 0.7426 - val_loss: 0.5316 - val_acc: 0.7302
Epoch 5/50
20000/20000 [==============================] - 59s 3ms/step - loss: 0.4976 - acc: 0.7564 - val_loss: 0.5192 - val_acc: 0.7486
Epoch 6/50
20000/20000 [==============================] - 57s 3ms/step - loss: 0.4736 - acc: 0.7719 - val_loss: 0.5279 - val_acc: 0.7258
Epoch 7/50
20000/20000 [==============================] - 57s 3ms/step - loss: 0.4549 - acc: 0.7813 - val_loss: 0.5504 - val_acc:

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 20, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12800)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 38403     
_________________________________________________________________
dropout_1 (Dropout)          (None, 3)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 8         
Total params: 38,731
Trainable params: 38,731
Non-trainable params: 0
_________________________________________________________________


In [18]:
#cifar
model_v2 = Sequential()
model_v2.add(Conv2D(32,kernel_size=(5, 5), activation= 'relu',  input_shape=X.shape[1:]))
model_v2.add(Conv2D(32,kernel_size=(5, 5), activation= 'relu',  input_shape=X.shape[1:]))
model_v2.add(MaxPooling2D(pool_size=(2, 2),strides=2))
model_v2.add(Dropout(0.2))

model_v2.add(Conv2D(16,kernel_size=(5, 5), activation= 'relu',  input_shape=X.shape[1:]))
model_v2.add(Conv2D(16,kernel_size=(5, 5), activation= 'relu',  input_shape=X.shape[1:]))
model_v2.add(MaxPooling2D(pool_size=(2, 2),strides=2))
model_v2.add(Dropout(0.2))

model_v2.add(Flatten())

model_v2.add(Dense(1024,activation='relu'))

model_v2.add(Dropout(0.2))
model_v2.add(Dense(1))
model_v2.add(Activation('sigmoid'))
#model_v2.add(Dense(2,activation='sigmoid',input_shape= (n_cols,) ))

In [19]:
model_v2.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
batch_size = 16

# this is the configuration for training set 
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# For testing only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator, will read pictures found in subfolers of 'data/train'
data_generator = train_datagen.flow_from_directory(
        './train/',  # this is the target directory
        target_size=(64, 64),  # all images will be resized to 150x150
        color_mode = 'rgb', # rgb, grayscale
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels


# Train model on dataset
model_v2.fit_generator(generator=data_generator,
                    #validation_data=validation_generator,
                    epochs=50,
                    callbacks =  [my_callback],
                    use_multiprocessing=True,
                    workers=3)

#data_generator

Found 25000 images belonging to 2 classes.
Epoch 1/50
 249/1563 [===>..........................] - ETA: 10:21 - loss: 0.7215 - acc: 0.5246

In [13]:
# https://rstudio-pubs-static.s3.amazonaws.com/279500_57cad8c546724b40ad3e90692f716ae4.html
model_v2.fit(X, y,
        batch_size=150, 
        epochs=50,
        validation_split=0.2,
        callbacks =  [my_callback])
model.save_weights('second_try.h5')  

Train on 20000 samples, validate on 5000 samples
Epoch 1/50
20000/20000 [==============================] - 516s 26ms/step - loss: 0.7185 - acc: 0.5222 - val_loss: 0.6826 - val_acc: 0.5426
Epoch 2/50
20000/20000 [==============================] - 505s 25ms/step - loss: 0.6775 - acc: 0.5952 - val_loss: 0.7867 - val_acc: 0.5874
Epoch 3/50
 5700/20000 [=======>......................] - ETA: 6:05 - loss: 0.6279 - acc: 0.6488

KeyboardInterrupt: 